In [3]:
import sys
import os
import warnings
import time
import numpy as np
import tensorflow as tf
import SimpleITK as sitk

sys.path.append("../")
warnings.filterwarnings('ignore')
from deepneuroan.models import ChannelwiseConv3D, ChannelwiseMaxpool3D

In [2]:
'''''
To test how to initialize weights with tensorflow
'''''
in_tensor = tf.random.uniform(shape=(2, 5, 5, 3), minval=0, maxval=1, seed=0)
f = np.stack([np.ones((1,1))]*3, axis=2)
f = np.stack([f*0, f*1], axis=3)

out_tensor = tf.nn.conv2d(
    input=in_tensor,
    filters=f, 
    strides=[1, 1, 1, 1],
    padding="VALID")
# Construct a `Session` to execute the graph.
sess = tf.compat.v1.Session()
# Execute the graph and store the value that `e` represents in `result`.
split_inputs = tf.split(in_tensor, in_tensor.shape[-1], axis=-1)

# check if in_tensor result[2] is indeed splitted or not result[1]
print(in_tensor[0,:,:,0])
print(split_inputs[0][0,:,:,0])
# check if convolution sum the channels or not
print(np.sum( np.sum(in_tensor[0,:,:,:], axis=2) ) == np.sum( out_tensor[0,:,:,1] ))
print(out_tensor[0,:,:,1])
print(f.shape)

tf.Tensor(
[[0.10086262 0.04828131 0.844468   0.6552025  0.37789786]
 [0.4662125  0.7741407  0.97539485 0.79527795 0.94884145]
 [0.36522734 0.20397687 0.3132056  0.45779288 0.04997289]
 [0.65840876 0.42018986 0.25117612 0.14169264 0.6759862 ]
 [0.30540073 0.9572604  0.6609794  0.35055983 0.8883625 ]], shape=(5, 5), dtype=float32)
tf.Tensor(
[[0.10086262 0.04828131 0.844468   0.6552025  0.37789786]
 [0.4662125  0.7741407  0.97539485 0.79527795 0.94884145]
 [0.36522734 0.20397687 0.3132056  0.45779288 0.04997289]
 [0.65840876 0.42018986 0.25117612 0.14169264 0.6759862 ]
 [0.30540073 0.9572604  0.6609794  0.35055983 0.8883625 ]], shape=(5, 5), dtype=float32)
True
tf.Tensor(
[[1.9197936  0.874279   1.7715013  2.0103426  2.064529  ]
 [2.154733   1.7008486  2.0440488  2.459166   2.1362352 ]
 [1.8843015  0.84527624 1.6288254  1.520882   1.3406955 ]
 [2.0531244  1.1880996  2.0071173  1.3580973  1.431062  ]
 [0.9358636  1.642369   1.1738203  1.6314249  2.2915263 ]], shape=(5, 5), dtype=float32)

In [ ]:
'''''
To test how to initialize weights with keras
'''''
in_tensor = tf.random.uniform(shape=(2, 5, 5, 3), minval=0, maxval=1, seed=0)
f = np.stack([np.ones((1,1))]*3, axis=2)
f = np.stack([f*0, f*1], axis=3)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(input_shape=(5, 5, 3), filters=2, kernel_size=(1,1), weights=[f,np.zeros(2)], activation='relu'))
res_keras = model.predict(in_tensor, steps=2)

print(model.get_weights())
print(res_keras[0,:,:,1])

In [ ]:
'''''
Depthwise conv3d
'''''
in_tensor = tf.random.uniform(shape=(2, 5, 5, 5, 3), minval=0, maxval=1, seed=0)
f = np.stack([np.ones((1, 1, 1))]*3, axis=3)
f = np.stack([f*0, f*1], axis=4)
print(f.shape)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv3D(input_shape=(5, 5, 5, 3), filters=2, kernel_size=(1, 1, 1), weights=[f,np.zeros(2)], activation='relu'))
res_keras = model.predict(in_tensor, steps=1)

# print(np.sum(sess.run(in_tensor)[1,], axis=3))
# res_keras[1,:,:,:,1]

In [ ]:
in_tensor = tf.random.uniform(shape=(2, 220, 220, 220, 3), minval=0, maxval=1, seed=0)
f = np.ones((1, 1, 1, 1))
fa = np.stack([f*0, f*1], axis=4)
f = np.ones((1, 1, 1, 2))
fb = np.stack([f*0, f*1, f*2, f*3], axis=4)

inp = tf.keras.Input(shape=(220, 220, 220, 3))
cw_conv3da = ChannelwiseConv3D(filters=2, kernel_size=(1, 1, 1), weights=[fa,np.zeros(2)])(inp)
cw_conv3db = ChannelwiseConv3D(filters=4, kernel_size=(1, 1, 1), weights=[fb,np.zeros(4)])(cw_conv3da)
model = tf.keras.Model(inputs=inp, outputs=[cw_conv3da, cw_conv3db])
res = model.predict(in_tensor, steps=3)
res_conv3da = res[0]
res_conv3db = res[1]

# # Test conv3da ( should be the same because filter is one for 2nd filter, 0 for the 1st filter)
# print(sess.run(in_tensor)[0,][:,:,:,2])
# print(res_conv3da[0,:,:,:,1,2])
# Test conv3db ( should be the sum of res_conv3da for 1st filter (in each channel), 2nd filter is the first one multiplied by two etc..., 0 for the 1st filter)
# print(np.sum(res_conv3da[1,:,:,:,:,1], axis=3))
# print(res_conv3db[1,:,:,:,1,1])
# print(3*np.sum(res_conv3da[0,:,:,:,:,0], axis=3))
# print(res_conv3db[0,:,:,:,3,0])
# print(np.sum(res_conv3db[0,]))
# print(np.sum(res_conv3db[2,]))
# print(np.sum(res_conv3db[4,]))

In [26]:
'''''
Test gaussian filtering and laplacian
'''''

def save_array_to_itk(data, name, template_dir):
    template = sitk.ReadImage(template_dir + "/template_on_grid.nii.gz", sitk.sitkFloat64)
    vol = sitk.GetImageFromArray(data)
    vol.SetOrigin(template.GetOrigin())
    vol.SetSpacing(tuple(2*np.array(template.GetSpacing())))
    vol.SetDirection(template.GetDirection())
    sitk.WriteImage(vol, "/home/ltetrel/Documents/work/DeepNeuroAN/notebooks/" + name + ".nii.gz")
    

x = np.empty((1, 220, 220, 220, 2), dtype=np.float64)
data_dir = "/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data"

# Preprocess template
template = sitk.ReadImage(data_dir + "/template_on_grid.nii.gz", sitk.sitkFloat64)
volume_data = sitk.GetArrayFromImage(template)
# volume_data = (volume_data - np.mean(volume_data)) / np.std(volume_data)
template_preprocessed = sitk.GetImageFromArray(volume_data)
template_preprocessed.SetOrigin(template.GetOrigin())
template_preprocessed.SetSpacing(template.GetSpacing())
template_preprocessed.SetDirection(template.GetDirection())
sitk.WriteImage(template_preprocessed, "/home/ltetrel/Documents/work/DeepNeuroAN/notebooks/template_input.nii.gz")

# Preprocess source image
img = sitk.ReadImage(data_dir + "/ses-vid001_task-video_run-01_bold_vol-0127_transfo-000001.nii.gz", sitk.sitkFloat64)
img_data = sitk.GetArrayFromImage(img)
# img_data = (img_data - np.mean(img_data)) / np.std(img_data)
img_preprocessed = sitk.GetImageFromArray(img_data)
img_preprocessed.SetOrigin(img.GetOrigin())
img_preprocessed.SetSpacing(img.GetSpacing())
img_preprocessed.SetDirection(img.GetDirection())
sitk.WriteImage(img_preprocessed, "/home/ltetrel/Documents/work/DeepNeuroAN/notebooks/source_input.nii.gz")

# creating input for keras
x[0, :, :, :, 0] = volume_data
x[0, :, :, :, 1] = img_data

f = np.zeros((3, 3, 3, 1, 1))
f[:, :, 0, 0, 0] = (1/16)*np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
f[:, :, 1, 0, 0] = (1/16)*np.array([[2, 4, 2], [4, 4, 4], [2, 4, 2]])
f[:, :, 2, 0, 0] = (1/16)*np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])

f2 = (-1)*np.ones((3, 3, 3, 1, 1))
f2[1, 1, 1,] = 8
f2[:, :, 0, 0, 0] = np.array([[0, 0, 0], [0, -1, 0], [0, 0, 0]])
f2[:, :, 1, 0, 0] = np.array([[0, -1, 0], [-1, 6, -1], [0, -1, 0]])
f2[:, :, 2, 0, 0] = np.array([[0, 0, 0], [0, -1, 0], [0, 0, 0]])

inp = tf.keras.Input(shape=(220, 220, 220, 2))
gauss = ChannelwiseConv3D(filters=1, strides=(2, 2, 2), kernel_size=(3, 3, 3), weights=[f, np.zeros(1)])(inp)
laplacian = ChannelwiseConv3D(filters=1, strides=(1, 1, 1), kernel_size=(3, 3, 3), weights=[f2, np.zeros(1)])(gauss)
# testing if ChannelwiseConv3D same as tf.conv3d
split_inputs = tf.split(inp, inp.shape[-1], axis=-1)
# conv = tf.keras.layers.Conv3D(filters=1, strides=(2, 2, 2), dilations=(2, 2, 2), kernel_size=(3, 3, 3), weights=[f, np.zeros(1)])
conv_target = tf.nn.conv3d(input=split_inputs[0], strides=(1, 2, 2, 2, 1), dilations=(1, 2, 2, 2, 1), filters=f, padding="VALID")
conv_source = tf.nn.conv3d(input=split_inputs[1], strides=(1, 2, 2, 2, 1), dilations=(1, 2, 2, 2, 1), filters=f, padding="VALID")
gauss_tf_conv = tf.keras.layers.Concatenate()([conv_target, conv_source])
# results
model = tf.keras.Model(inputs=inp, outputs=[gauss, laplacian, gauss_tf_conv])
res = model.predict(x, steps=1)

# #benchmarking channelsize vs tf conv
# tic = time.time()
# for i in range(10):
#     m = tf.keras.Model(inputs=inp, outputs=[gauss])
#     r = m.predict(x, steps=1)
# ElpsTime = time.time() - tic
# print("%1.2f s"%(ElpsTime))

# tic = time.time()
# for i in range(10):
#     m = tf.keras.Model(inputs=inp, outputs=[gauss_tf_conv])
#     r = m.predict(x, steps=1)
# ElpsTime = time.time() - tic
# print("%1.2f s"%(ElpsTime))

save_array_to_itk(res[0][0, :, :, :, 0, 0], "template_gaussian", data_dir)
save_array_to_itk(res[1][0, :, :, :, 0, 0], "template_laplace", data_dir)
save_array_to_itk(res[2][0, :, :, :, 0], "template_gauss_tf", data_dir)
save_array_to_itk(res[0][0, :, :, :, 0, 1], "source_gaussian", data_dir)
save_array_to_itk(res[1][0, :, :, :, 0, 1], "source_laplace", data_dir)
save_array_to_itk(res[2][0, :, :, :, 1], "source_gaussian_tf", data_dir)

W0131 11:41:46.481798 140535902078784 training_utils.py:1212] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


InvalidArgumentError:  CPU implementation of Conv3D currently only supports dilated rates of 1.
	 [[node Conv3D_12 (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_1887]

Function call stack:
keras_scratch_graph


In [24]:
res[2].shape

(1, 109, 109, 109, 2)

In [ ]:
'''''
Test maxpool with reshape
Input (batch, height, width, depth, filters, channels)
Output should be (batch, new_height, new_width, new_depth, filters, channels)
'''''
in_tensor = tf.random.uniform(shape=(2, 5, 5, 5, 2, 3), minval=0, maxval=1, seed=0)

tic = time.time()

inp = tf.keras.Input(shape=(5, 5, 5, 2, 3))
# new_shape = tuple(inp.shape[:-2]) + (int(inp.shape[-2]*inp.shape[-1]),)
# print(inp.shape.as_list())
if(len(inp.shape.as_list()) > 5):
    new_shape = [-1] + inp.shape.as_list()[1:4] + [inp.shape.as_list()[4]*inp.shape.as_list()[5]]
    squeezed = tf.keras.backend.reshape(x=inp, shape=new_shape)
    cw_pool3da = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), padding="VALID")(squeezed)
    output = tf.keras.backend.reshape(x=cw_pool3da, shape=[-1, 2, 2, 2, 2, 3])
model = tf.keras.Model(inputs=inp, outputs=[output])

res = model.predict(in_tensor, steps=1000)

ElpsTime = time.time() - tic
print("%1.2f s"%(ElpsTime))

# print("input")
# print(sess.run(in_tensor)[0, :, :, :, 0, 0])
# print("output")
# print(res[0, :, :, :, 0, 0])